The following code will create some helper functions to load entity and relation mappings to IDs and the relational tuples that map between subject-relation-object entities. All these are tab-separated and provided by the FinDKG project dataset. See https://xiaohui-victor-li.github.io/FinDKG for more details.

In [1]:
def load_entities(filename):
    lines = open(filename).readlines()
    all_entities = {}
    for line in lines:
        entity, id, _, count = line.strip().split("\t")
        all_entities[int(id)] = entity
    
    return all_entities

def load_relations(filename):
    lines = open(filename).readlines()
    all_relations = {}
    for line in lines:
        relation, id = line.strip().split("\t")
        all_relations[int(id)] = relation
    
    return all_relations

def get_relation_tuples(all_entities, all_relations, dataset):
    # load the data
    lines = open(dataset).readlines()
    all_tuples = []
    for line in lines:
        subject, relation, obj, time, _ = line.strip().split("\t")
        all_tuples.append((all_entities[int(subject)], all_relations[int(relation)], all_entities[int(obj)]))
    return all_tuples

In [2]:
ENTITY_ID_MAP = "dkg_gnn/data/FinDKG-full/entity2id.txt"
RELATION_ID_MAP = "dkg_gnn/data/FinDKG-full/relation2id.txt"
# test/train data: The first four columns correspond to subject (entity), relation, object (entity), and time.
DATASET = "dkg_gnn/data/FinDKG-full/test.txt"

all_entities = load_entities(ENTITY_ID_MAP)
all_relations = load_relations(RELATION_ID_MAP)
knowledge_graph = get_relation_tuples(all_entities, all_relations, DATASET)

Now that we have the entities, relations and the entire knowledge graph in English, let's ingest it into NetworkX for visualization. First install networkX using the command below. 

In [ ]:
%pip install --upgrade --quiet  networkx langchain

Let's build the network query chain using LangChain following this tutorial. https://python.langchain.com/docs/use_cases/graph/graph_networkx_qa. For this, we already have the relation/entity triplets, so let's add them to a new networkx graph manually.

In [6]:
from langchain.indexes.graph import NetworkxEntityGraph
from langchain.graphs.networkx_graph import KnowledgeTriple

graph = NetworkxEntityGraph()
for item in knowledge_graph:
    kt = KnowledgeTriple(item[0], item[1], item[2])
    graph.add_triple(kt)

Let's check what sort of triples have been generated using the graph.get_triples() function.

In [7]:
graph.get_triples()

[('Companies', 'IPO Market', 'Participates_In'),
 ('Companies', 'Labor Shortages', 'Has'),
 ('Companies', 'Corporate bonds', 'Raise'),
 ('Companies', 'Silicon Valley', 'Operate_In'),
 ('Companies', 'Digital Markets Act', 'Is_Member_Of'),
 ('Companies', 'Food and Drug Administration', 'Has'),
 ('Bank of England', 'Inflation', 'Impact'),
 ('Bank of England', 'United Kingdom', 'Control'),
 ('Bank of England', 'Interest Rates', 'Control'),
 ('Bank of England', 'U.K. Government Bonds', 'Negative_Impact_On'),
 ('Bank of England', 'Financial markets', 'Control'),
 ('Bank of England', 'Bond market', 'Control'),
 ('Bank of England', 'Government bonds', 'Control'),
 ('Bank of England', '10-year Treasury Note', 'Control'),
 ('Bank of England', 'Bank of England', 'Relate_To'),
 ('Bank of England', 'Government bond yields', 'Relate_To'),
 ('Bank of England', 'Monetary Policy', 'Impact'),
 ('Bank of England', 'Corporate bonds', 'Raise'),
 ('Bank of England', 'Borrowing Cost', 'Raise'),
 ('Bank of En

Looks pretty great! Let's try querying this graph by using an LLM chain. We will use the free NVIDIA AI Endpoints Mixtral-8x7B model (https://catalog.ngc.nvidia.com/orgs/nvidia/teams/ai-foundation/models/mixtral-8x7b/api). We can access them through the LangChain connector: https://python.langchain.com/docs/integrations/chat/nvidia_ai_endpoints.

In [ ]:
%pip install --quiet langchain_nvidia_ai_endpoints

In [9]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain.chains import GraphQAChain
NVIDIA_API_KEY = "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" #from https://catalog.ngc.nvidia.com/orgs/nvidia/teams/ai-foundation/models/mixtral-8x7b/api
llm = ChatNVIDIA(model="mixtral_8x7b", nvidia_api_key=NVIDIA_API_KEY)

In [15]:
# llm.invoke("hello")
chain = GraphQAChain.from_llm(llm = llm, graph=graph, verbose=True)
res = chain.run("Explain how natural gas prices in Saudi Arabia could impact the US economy.")
print(res)



> Entering new GraphQAChain chain...
Entities Extracted:
Saudi Arabia, natural gas prices, US economy
Full Context:
Saudi Arabia Has Crown Prince Mohammed Bin Salman
Saudi Arabia Has Public Investment Fund
Saudi Arabia Negative_Impact_On Oil prices
Saudi Arabia Impact Inflation
Saudi Arabia Relate_To OPEC
Saudi Arabia Relate_To King Salman
Saudi Arabia Operate_In Russia
Saudi Arabia Produce OPEC+
Saudi Arabia Operate_In Germany
Saudi Arabia Relate_To United States
Saudi Arabia Operate_In Middle East
Saudi Arabia Operate_In oil industry
Saudi Arabia Control Oil production
Saudi Arabia Operate_In Oil market
Saudi Arabia Relate_To Europe
Saudi Arabia Relate_To Rising Food and Energy Prices
Saudi Arabia Relate_To Xi Jinping
Saudi Arabia Invests_In Ballistic Missiles

> Finished chain.
I don't have real-time information or access to current data, but I can provide a general explanation based on the given knowledge triplets.

Saudi Arabia is related to the US in various ways, including ope

Feel free to try more examples - as with all LLM projects, prompt engineering is important to capture the right relationships. Another good way to understand the graph better is to create a visualization. Let's do this using PyVis, since it has good integration with networkx.

In [ ]:
%pip install --quiet --upgrade pyvis

In [ ]:
from pyvis.network import Network
import networkx as nx
graph.write_to_gml("graph.gml")
G = nx.read_gml("graph.gml")
nt = Network(notebook=True, cdn_resources='in_line')
nt.from_nx(G)
nt.show("network.html")

![[Image 1] Zoomed-in knowledge graph"](images/kg1.png "[Image 1] Zoomed-in knowledge graph")
![[Image 2] Zoomed-in knowledge graph"](images/kg2.png "[Image 2] Zoomed-in knowledge graph")